In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [32]:
df = pd.read_csv('data/train_data_fix_concat.csv')
df_test = pd.read_csv('data/test_data_fix_concat.csv')
df.head(3)

,oil_type,blend_id,oil_property_param_title,oil_property_param_value,component_name,component_class,polymer,component_property_param_title,component_property_param_value,smiles
0,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,02236ee6-5eec-4368-a2e4-6f2e73fb0f96,0.0000,CCCCC
1,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,2511714c-ab50-4566-bc92-8e4095d87d01,0.0000,CCCCC
2,3fa07e0a-415c-496d-b88b-557855cb3e77,49743a76-a614-11ee-9529-005056921581,4c7a51f1-dc82-41dc-92fa-772535c2c70c,15.58,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,no,38b06d9a-bcf2-40de-8e1b-9c3988c42406,0.0001,CCCCC


In [33]:
df.isna().sum()

oil_type                            1943
blend_id                               0
oil_property_param_title               0
oil_property_param_value            2344
component_name                         0
component_class                   141105
polymer                           141105
component_property_param_title         0
component_property_param_value    238777
smiles                            141105
dtype: int64

Заполним пропуски в данных тупыми значениями

In [34]:
df['oil_type'] = df['oil_type'].fillna(value='unknown')
df['oil_property_param_value'] = df['oil_property_param_value'].fillna(value=-1)
df['component_class'] = df['component_class'].fillna(value='unknown')
df['component_property_param_value'] = df['component_property_param_value'].fillna(value=-1)
df['smiles'] = df['smiles'].fillna(value='unknown')
df['polymer'] = df['polymer'].fillna(value='unknown')

И в тесте тоже

In [20]:
df_test['oil_type'] = df_test['oil_type'].fillna(value='unknown')
df_test['oil_property_param_value'] = df_test['oil_property_param_value'].fillna(value=-1)
df_test['component_class'] = df_test['component_class'].fillna(value='unknown')
df_test['component_property_param_value'] = df_test['component_property_param_value'].fillna(value=-1)
df_test['smiles'] = df_test['smiles'].fillna(value='unknown')
df_test['polymer'] = df_test['polymer'].fillna(value='unknown')

Сделаем список из blend_id в train датасете у которых в свойствах есть наш таргет-свойство

In [35]:
blend_ids_with_target = df[df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8']['blend_id'].unique().tolist()

In [36]:
df = df[df['blend_id'].isin(blend_ids_with_target)].reset_index(drop=True)
# Оставим бленды где в свойствах есть наш таргет (Потеряли немного строк)

Создаем таблицу с колонками component_property

In [37]:
pivot_df = df.pivot_table(index=['oil_type', 'blend_id', 'oil_property_param_title', 'oil_property_param_value', 'component_name', 'component_class', 'smiles'],
                          columns='component_property_param_title',
                          values='component_property_param_value',
                          fill_value=np.nan).reset_index()

# Rename columns
pivot_df.columns.name = None

Также в тесте

In [38]:
pivot_df_test = df_test.pivot_table(index=['oil_type', 'blend_id', 'oil_property_param_title', 'oil_property_param_value', 'component_name', 'component_class', 'smiles'],
                          columns='component_property_param_title',
                          values='component_property_param_value',
                          fill_value=np.nan).reset_index()

# Rename columns
pivot_df_test.columns.name = None

In [39]:
pivot_df.head(3)

,oil_type,blend_id,oil_property_param_title,oil_property_param_value,component_name,component_class,smiles,009c5608-34ee-44ec-a294-a6252d691af3,02236ee6-5eec-4368-a2e4-6f2e73fb0f96,0817d2c9-2478-40d2-aab3-6b055f8071f0,...,c11112c4-487f-4e2c-812d-84a29b240b69,d037c9ef-a6e1-4328-832a-9db50bbb92af,d12a35d5-14a3-4828-9c59-bff8e912034d,dc511668-8a45-42ae-ab39-aeadd5f3a0f3,dc908e5f-ccb4-44cb-a605-985ebf5b0d1c,de405dfd-ef89-4b81-80f0-c25147817cdc,e1e7b28a-e529-4e7a-8d60-3e24178ac9fe,e3983d82-8773-4e46-8f88-147098d7b87a,f216a0c7-d453-4b15-9b4a-7647cbe2d874,f400be8b-3bed-4f17-8a12-9fabd9b73586
0,3fa07e0a-415c-496d-b88b-557855cb3e77,007b4d7a-b1f8-11ed-93b4-005056921581,049e5cb9-9c20-438c-9ef2-96870712a0be,0.016,0444c588-f816-4b9b-a6dc-f596b021d6e3,базовое масло 2 гр,CC(C)CCCCC,200.0,NaN,-1.0,...,-1.00,-1.0,NaN,NaN,NaN,NaN,-1.0,90.0,-1.000,-1.0
1,3fa07e0a-415c-496d-b88b-557855cb3e77,007b4d7a-b1f8-11ed-93b4-005056921581,049e5cb9-9c20-438c-9ef2-96870712a0be,0.016,29c23be0-273f-490d-b1dd-5a09c45eccf3,unknown,unknown,NaN,NaN,-1.0,...,3.35,-1.0,NaN,NaN,NaN,NaN,-1.0,NaN,-1.000,-1.0
2,3fa07e0a-415c-496d-b88b-557855cb3e77,007b4d7a-b1f8-11ed-93b4-005056921581,049e5cb9-9c20-438c-9ef2-96870712a0be,0.016,615537f6-1f8f-4240-a5e9-8f7be344ecd3,базовое масло 1 гр,CCCCC,-1.0,0.0,NaN,...,NaN,NaN,40.0,-17.0,NaN,NaN,NaN,14.0,0.017,NaN


In [40]:
print(f'Размер датасета train: {pivot_df.shape}')
print(f'Размер датасета test: {pivot_df_test.shape}')

Размер датасета train: (16797, 50)
Размер датасета test: (3684, 35)


В данный момент в наших смесях из трейн датасета здесь есть свойство-таргет, значение которого нужно занести в новую колонку

Посмотрим на значения этого свойства для каждого бленда

In [41]:
pivot_df[pivot_df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8'][['blend_id', 'oil_property_param_value']].drop_duplicates().head(2)

,blend_id,oil_property_param_value
42,007b4d7a-b1f8-11ed-93b4-005056921581,13360.0
70,04aed446-6beb-11ed-87f9-005056921581,7416.0


In [43]:
targets_values = pivot_df[pivot_df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8'][['blend_id', 'oil_property_param_value']].drop_duplicates()

In [44]:
merged_df = pd.merge(pivot_df, targets_values, on='blend_id', how='left')

In [46]:
merged_df = merged_df[merged_df['oil_property_param_title'] != 'ad7e6027-00b8-4c27-918c-d1561f949ad8']
# Удалим строчки со свойствами-таргета

In [47]:
merged_df = merged_df.rename(columns={'oil_property_param_value_y': 'target', 'oil_property_param_value_x': 'oil_property_param_value'})

In [65]:
merged_df.shape

(14716, 51)

Добавим колонку target заполненную nan к pivot_df_test

In [51]:
pivot_df_test['target'] = np.nan

In [52]:
final_df = pd.concat([merged_df, pivot_df_test], ignore_index=True)

In [54]:
final_df.shape

(18400, 67)

In [61]:
final_df.isna().sum().to_dict()

{'oil_type': 0,
 'blend_id': 0,
 'oil_property_param_title': 0,
 'oil_property_param_value': 0,
 'component_name': 0,
 'component_class': 0,
 'smiles': 0,
 '009c5608-34ee-44ec-a294-a6252d691af3': 12143,
 '02236ee6-5eec-4368-a2e4-6f2e73fb0f96': 12697,
 '0817d2c9-2478-40d2-aab3-6b055f8071f0': 9809,
 '08db8d40-6b81-44ab-ab91-4b482ebc4187': 11641,
 '0bede6ba-5118-4a24-8c61-4df27f5746d7': 18002,
 '0ec6a170-4283-4466-8e16-ab71675053d6': 9809,
 '1931ea10-9783-4065-a77c-04532d7d0559': 9809,
 '1a8ce77b-9eac-489d-b6fc-4d7df0b74092': 18321,
 '1da38bd4-1cca-467b-b480-76c4fac65d5f': 9809,
 '2511714c-ab50-4566-bc92-8e4095d87d01': 12697,
 '266a1a34-14d1-49f7-b1ca-d354f9ff6ce8': 9809,
 '2a6e6683-e8f2-49ae-8114-b752a1e97c60': 9809,
 '2caf7dfa-de92-43ba-811d-ce9bbac9443a': 18345,
 '38b06d9a-bcf2-40de-8e1b-9c3988c42406': 4108,
 '3f7871a6-3ebc-4d7c-a5c4-3867820048e1': 18343,
 '5495e254-d5be-40ab-a96a-bb5da1c4da57': 9809,
 '594a3970-ec4a-48bb-bd37-c00e2ce90d83': 11641,
 '59fae799-21eb-4093-b36a-5a9dc69119f

In [62]:
train_df = final_df[~pd.isna(final_df['target'])]
test_df = final_df[pd.isna(final_df['target'])] 

In [64]:
train_df.shape

(14716, 67)

In [66]:
train_df.to_csv('data/train_preprocessed.csv', index=False)
test_df.to_csv('data/test_preprocessed.csv', index=False)